In [4]:
import laspy
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import os

file_path = "C:\\Users\\patel1637\\Downloads\\Unitconversion\\SK_LastMountainLake_202106_NAD83CSRS_UTM13N_1km_N452_E5719_CLASS.copc.las"
download_path = os.path.expanduser("~\\Downloads")  # Get the path to the Downloads directory
chunk_size = 1000000  # Number of points to process at a time

with laspy.open(file_path, mode='r', laz_backend='laszip') as las_file:
    print(las_file.header.point_count)
    
    las = las_file.read()
    print('Points from data:', len(las.points))

    # Save DataFrame to Excel in the Downloads directory
    excel_file_path = os.path.join(download_path, "points2.xlsx")
    writer = pd.ExcelWriter(excel_file_path, engine='openpyxl')

    # Check if the Excel file already exists
    if os.path.isfile(excel_file_path):
        book = openpyxl.load_workbook(excel_file_path)
        writer.book = book

    # Create a new workbook if the file doesn't exist
    else:
        writer.book = Workbook()

    # Check if 'Sheet1' already exists in the workbook
    sheet_name = 'Sheet1'
    if sheet_name in writer.book.sheetnames:
        writer.sheets = {ws.title: ws for ws in writer.book.worksheets}
        sheet = writer.book[sheet_name]
    else:
        sheet = writer.book.create_sheet(title=sheet_name)
        writer.sheets = {ws.title: ws for ws in writer.book.worksheets}

    # Create a list to store the point coordinates
    points_list = []

    # Extract X, Y, and Z coordinates
    for i in range(len(las.points)):
        x = las.points[i]['X']
        y = las.points[i]['Y']
        z = las.points[i]['Z']
        points_list.append([x, y, z])

        if len(points_list) == chunk_size:
            # Convert the points list to a DataFrame
            df = pd.DataFrame(points_list, columns=['X', 'Y', 'Z'])

            # Write DataFrame to Excel file in chunks
            for row in dataframe_to_rows(df, index=False, header=True):
                sheet.append(row)
            
            # Clear points_list for the next chunk
            points_list.clear()

    # Process any remaining points
    if len(points_list) > 0:
        df = pd.DataFrame(points_list, columns=['X', 'Y', 'Z'])
        for row in dataframe_to_rows(df, index=False, header=True):
            sheet.append(row)

    # Save and close the Excel file
    writer.save()
    writer.close()

    print(f"Excel file saved at: {excel_file_path}")


8301200
Points from data: 8301200


BadZipFile: File is not a zip file